In [ ]:
from pyspark.sql import SparkSession

In [ ]:
class LazySparkSession:
    packages = [
        "io.delta:delta-spark_2.13:4.0.0",
        "org.apache.hadoop:hadoop-aws:3.4.0",
    ]

    def __init__(self, access_key, secret_key, endpoint):
        self._access_key = access_key
        self._secret_key = secret_key
        self._endpoint = endpoint
        

    def start(
        self,
        app_name: str = "Airflow Spark Delta Minio App",
        executor_memory: str = "1g",
        driver_memory: str = "1g",
        driver_maxresultsize: str = "1g",
        master_url: str = "local[*]",
    ):

        builder = (
            SparkSession
            .Builder()
            .appName(app_name)
            .config("spark.hadoop.fs.s3a.access.key", self._access_key)
            .config("spark.hadoop.fs.s3a.secret.key", self._secret_key)
            .config("spark.hadoop.fs.s3a.endpoint", self._endpoint)
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
            .config("spark.hadoop.fs.s3a.path.style.access", "true")
            #
            .config("spark.hadoop.delta.enableFastS3AListFrom", "true")
            #
            # .config("spark.hadoop.hive.metastore.uris", "thrift://127.0.0.1:9083")
            #
            .config("spark.executor.memory", executor_memory)
            #
            .config("spark.driver.memory", driver_memory)
            .config("spark.driver.maxResultSize", driver_maxresultsize)
            #
            # .config("spark.sql.warehouse.dir", "s3a://lakehouse")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            # .config("spark.sql.hive.metastore.version", "4.0.0")
            # .config("spark.sql.hive.metastore.jars", "lib/*")
            #
            .config("spark.jars.packages", ",".join(self.packages))
            #
            .master(master_url)
            # .enableHiveSupport()
        )

        return builder.getOrCreate()

In [ ]:
spark = LazySparkSession(
    access_key="9a5edc7df42cfd71", 
    secret_key="DW4CyCit2CGO9gXyRtqQ7Lj6ZzQZm3xfS33ItBQpVbc=", 
    endpoint="http://127.0.0.1:10000"
).start()

In [ ]:
df = spark.createDataFrame([(1,2), (1,2), (1,2)], schema="a integer, b integer")

In [ ]:
df.write.format("delta").mode('overwrite').save("s3a://bronze/numbers/integers")

In [ ]:
spark.read.format('delta').load('s3a://bronze/numbers/integers').show()

In [ ]:
# 25/08/21 07:54:12 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
# SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
# SLF4J: Defaulting to no-operation (NOP) logger implementation
# SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.

# 25/08/21 07:54:15 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
# Hive Session ID = e38db77a-6afe-437e-bc8e-d5fa7f632b38
# 25/08/21 07:54:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
# 25/08/21 07:54:51 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to numbers/integer/part-00000-8c5f40d3-4def-4580-b432-fc63efb209cc-c000.snappy.parquet. This is Unsupported
# 25/08/21 07:55:02 WARN HiveExternalCatalog: Could not alter schema of table `numbers`.`integer` in a Hive compatible way. Updating Hive metastore in Spark SQL specific format.

# Caused by: org.apache.spark.rpc.RpcEndpointNotFoundException: Cannot find endpoint: spark://CoarseGrainedScheduler@10.255.255.254:34253

# WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic

In [ ]:
# create catalog bronze using delta_lake with (
#     "hive.metastore.uri"='thrift://host.docker.internal:9083',
#     "s3.aws-access-key"='9a5edc7df42cfd71',
#     "s3.aws-secret-key"='DW4CyCit2CGO9gXyRtqQ7Lj6ZzQZm3xfS33ItBQpVbc=',
#     "s3.endpoint"='http://host.docker.internal:10000',
#     "s3.region"='us-east-1',
#     "fs.native-s3.enabled"='true',
#     "s3.path-style-access"='true',
#     "delta.metastore.store-table-metadata"='true',
#     "delta.register-table-procedure.enabled"='true',
#     "delta.deletion-vectors-enabled"='true',
#     "delta.enable-non-concurrent-writes"='true'
# )

# create schema if not exists number with (
#     "location" = 's3a://bronze/numbers'
# )

# CALL bronze.system.register_table(schema_name => 'numbers', table_name => 'integers', table_location => 's3://bronze/numbers/integergs')

In [ ]:
# Creating a Git Commit Message Convention for Your Team

# https://medium.com/@naandalist/creating-a-git-commit-message-convention-for-your-team-acb4b3edfc44